### Preprocess data

In [19]:
import os, glob
import numpy as np
import SimpleITK as sitk
import torch
import nibabel as nib
import shutil
import matplotlib.pyplot as plt
import torch.nn.functional as F
from skimage.exposure import equalize_hist
from einops.einops import rearrange

In [124]:
def getSymmetricRepresentation(ct_volume):
    # the volume is either h x w x t or h x w x modalities with the first modality the NCCT
    ct_slice = ct_volume[0]
    ct_slice_flipped = np.fliplr(ct_slice)
    fixedImage = sitk.GetImageFromArray(ct_slice)
    movingImage = sitk.GetImageFromArray(ct_slice_flipped)

    # Elastix part, determine rotation
    parameterMap = sitk.GetDefaultParameterMap("rigid")
    elastixImageFilter = sitk.ElastixImageFilter()
    elastixImageFilter.SetFixedImage(fixedImage)
    elastixImageFilter.SetMovingImage(movingImage)
    elastixImageFilter.LogToFileOn()
    elastixImageFilter.SetParameterMap(parameterMap)
    resultImage = elastixImageFilter.Execute()

    # We now determined the transformation of the flipped scan to the original,
    # either for the first frame or for the first modality
    # Now we apply the same transformation to the other modalities or the other frames

    empty = np.zeros((ct_volume.shape[0],256,256))
    for frame_nr in range(ct_volume.shape[0]):
        frame = ct_volume[frame_nr,:,:]
        frame_flipped = np.fliplr(frame)
        movingFrame = sitk.GetImageFromArray(frame_flipped)
        frameResult = sitk.Transformix(movingFrame, elastixImageFilter.GetTransformParameterMap())
        empty[frame_nr,:,:] = sitk.GetArrayFromImage(frameResult)
    return empty

def getSym(volume):
    empty = np.zeros((volume.shape[0],256,256,2))
    sym_frame = getSymmetricRepresentation(volume)
    print(sym_frame.shape)
    empty[...,0] = volume
    empty[...,1] = sym_frame
    return empty

def preprocessor(nifti, clip_value): #3D nifti
    # Clip entire 4D volume
    nifti = np.clip(nifti, 0, clip_value)
    # Equalize entire histogram
    nifti = equalize_hist(nifti, nbins=20000, mask=(nifti > 0))
    nifti = nifti - np.min(nifti)
    # Shift distribution, not the zeroes
    mask=(nifti > 0)
    mdata = np.ma.masked_array(nifti, mask=~mask.astype(bool))
    mdata =(mdata - 0.5)
    mdata.mask = np.ma.nomask
    return mdata

def smoothing(array):
    kernel = np.array([.25, .5, .25])
    # h w c t
    channel0 = array[:,:,0,:]
    channel1 = array[:,:,1,:]
    kernel = torch.tensor(kernel).to('cuda')
    kernel = kernel.view(1,1,3)
    empty = np.empty([256,256,2,array.shape[-1]//2])
    for i in range(256):
        for j in range(256):
            vector0 = torch.tensor(channel0[i,j,:]).to('cuda')
            vector0 = vector0.view(1,1,len(vector0))
            output0 = F.conv1d(vector0, kernel, stride=2, padding=0)
            output0 = output0.cpu().numpy()
            empty[i,j,0,:] = output0

            vector1 = torch.tensor(channel1[i,j,:]).to('cuda')
            vector1 = vector1.view(1,1,len(vector1))
            output1 = F.conv1d(vector1, kernel, stride=2, padding=0)
            output1 = output1.cpu().numpy()
            empty[i,j,1,:] = output1
    return empty

def write_np(array, path):
    with open(path, 'wb') as f:
        np.save(f, array)

In [132]:
def datamaker(case = 8, dataset='TRAINING'):
    path = os.path.join('ISLES2018',dataset)
    for file in glob.glob(os.path.join(path,f'case_{case}','*/*.nii')):
        if 'CT_MTT' in file:
            CT_MTT = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT_MTT = rearrange(CT_MTT, 'd h (w c) -> c h w d', c=1)
        if 'CT_Tmax' in file:
            CT_Tmax = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT_Tmax = rearrange(CT_Tmax, 'd h (w c) -> c h w d', c=1)
        if 'CT_CBF' in file:
            CT_CBF = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT_CBF = rearrange(CT_CBF, 'd h (w c) -> c h w d', c=1)
        if 'CT_CBV' in file:
            CT_CBV = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT_CBV = rearrange(CT_CBV, 'd h (w c) -> c h w d', c=1)
        if 'OT' in file and dataset=='TRAINING':
            CT_MASK = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT_MASK = rearrange(CT_MASK, 'd h w -> h w d')
        if 'CT_4DPWI' in file:
            CT_4DPWI = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT_4DPWI = rearrange(CT_4DPWI, ' t d h w -> t h w d')
        if '.CT.' in file:
            CT = sitk.GetArrayFromImage(sitk.ReadImage(file))
            CT = rearrange(CT, 'd h (w c) -> c h w d', c=1)
    print('Data case %i loaded'%(case))
    folder = 'train' if dataset == 'TRAINING' else 'test'
    os.makedirs(os.path.join(os.getcwd(), folder, 'COMPLETE_MASK'), exist_ok=True)
    maskpath = [file for file in glob.glob(os.path.join(path,f'case_{case}','*/*.nii')) if 'OT' in file][0]
    savename = os.path.join(os.getcwd(), folder, 'COMPLETE_MASK', 'case_{}.nii'.format(str(case).zfill(2)))
    shutil.copy(maskpath,savename)
    # create brainmask
    CT_SKULL = CT
    skull = CT_CBV + CT_MTT + CT_Tmax + CT_CBF
    skull = (skull>0)

    CT = np.multiply(skull,CT)
    # create CTP without skull
    skull_frames = np.zeros(CT_4DPWI.shape)
    for i in range(CT_4DPWI.shape[0]):
        skull_frames[0,...] = skull[0,...]
    CT_4DPWI  = np.multiply(skull_frames,CT_4DPWI)
    # we generate the data per slice
    for _z in range(CT.shape[-1]):
        # We only use slices with infarcts fro training
        if np.max(CT_MASK[:,:,_z]) != 1.0 and dataset=='TRAINING':
            print('Slice without infarct found, continuing')
            continue

        baseline = preprocessor(CT[:,:,:,_z], 500)
        mtt = preprocessor(CT_MTT[:,:,:,_z], 500)
        tmax = preprocessor(CT_Tmax[:,:,:,_z], 500)
        cbf = preprocessor(CT_CBF[:,:,:,_z], 500)
        cbv = preprocessor(CT_CBV[:,:,:,_z], 500)
        ct_with_skull = preprocessor(CT_SKULL[:,:,:,_z], 500)
        ctp = preprocessor(CT_4DPWI[:,:,:,_z], 500)
        ctp_sym = getSym(ctp) # h x w x c x t
        ctp_sym =  rearrange(ctp_sym, 't h w c -> h w c t')
        ctp_sym = smoothing(ctp_sym)
        ct_modalities = np.concatenate([baseline, mtt, tmax, cbf, cbv,ct_with_skull], axis=0)

        ct_modalities_sym = getSym(ct_modalities)

        ct_modalities_sym =  rearrange(ct_modalities_sym, 'mods h w c -> mods c h w')
        ctp_sym =  rearrange(ctp_sym, 'h w c t -> t c h w')

        folder = 'train' if dataset == 'TRAINING' else 'test'
        os.makedirs(os.path.join(os.getcwd(), folder, 'CTP'), exist_ok=True)
        os.makedirs(os.path.join(os.getcwd(), folder, 'CT'), exist_ok=True)
        os.makedirs(os.path.join(os.getcwd(), folder, 'CTP_CBF'), exist_ok=True)
        os.makedirs(os.path.join(os.getcwd(), folder, 'CTP_CBV'), exist_ok=True)
        os.makedirs(os.path.join(os.getcwd(), folder, 'CTP_MTT'), exist_ok=True)
        os.makedirs(os.path.join(os.getcwd(), folder, 'CTP_Tmax'), exist_ok=True)
        os.makedirs(os.path.join(os.getcwd(), folder, 'MASK'), exist_ok=True)
        write_np(ct_modalities_sym[0], os.path.join(os.getcwd(), folder, 'CT', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))
        write_np(ct_modalities_sym[1], os.path.join(os.getcwd(), folder, 'CTP_MTT', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))
        write_np(ct_modalities_sym[2], os.path.join(os.getcwd(), folder, 'CTP_Tmax', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))
        write_np(ct_modalities_sym[3], os.path.join(os.getcwd(), folder, 'CTP_CBF', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))
        write_np(ct_modalities_sym[4], os.path.join(os.getcwd(), folder, 'CTP_CBV', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))
        write_np(ctp_sym, os.path.join(os.getcwd(), folder, 'CTP', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))
        if dataset=='TRAINING':
            write_np(CT_MASK[:,:,_z], os.path.join(os.getcwd(), folder, 'MASK', 'case_%s_%s.npy'%(str(case).zfill(2),str(_z).zfill(2))))

In [133]:
for i in range(95):
    datamaker(case=i)

Data case 8 loaded
(1, 256, 256, 8)
(49, 256, 256, 8) (1, 256, 256, 8)
(49, 256, 256, 8) (1, 256, 256, 8)
(49, 256, 256)
(49, 256, 256)
(256, 256, 2, 49) (1, 256, 256)
(6, 256, 256) (1, 256, 256)
(6, 256, 256)
(6, 256, 256, 2) (1, 256, 256)
(49, 256, 256)
(49, 256, 256)
(256, 256, 2, 49) (1, 256, 256)
(6, 256, 256) (1, 256, 256)
(6, 256, 256)
(6, 256, 256, 2) (1, 256, 256)
(49, 256, 256)
(49, 256, 256)
(256, 256, 2, 49) (1, 256, 256)
(6, 256, 256) (1, 256, 256)
(6, 256, 256)
(6, 256, 256, 2) (1, 256, 256)
(49, 256, 256)
(49, 256, 256)
(256, 256, 2, 49) (1, 256, 256)
(6, 256, 256) (1, 256, 256)
(6, 256, 256)
(6, 256, 256, 2) (1, 256, 256)
Slice without infarct found, continuing
Slice without infarct found, continuing
Slice without infarct found, continuing
Slice without infarct found, continuing
